In [234]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener
from selenium.webdriver import Firefox
import time
from selenium.webdriver.common.action_chains import ActionChains

df=pd.DataFrame(columns=['route_name','route_link','bus_name','bus_type','departing_time','duration','reaching_time','star_rating','price','seats_available'])
df

,route_name,route_link,bus_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seats_available


In [241]:
travels_links_dict = {}

driver = webdriver.Firefox()
driver.get('https://www.redbus.in/')
driver.maximize_window()


WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'rtcName')))
popular_bus_depts = driver.find_elements(By.CLASS_NAME, 'rtcName')
view_buses_buttons = driver.find_element(By.XPATH, "/html/body/section/div[2]/main/div[3]/div[3]/div[1]/div[2]/a")
popular_bus_names = [bus.text for bus in popular_bus_depts] 
driver.get(view_buses_buttons.get_attribute('href'))
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'D113_link')))
bus_depts = driver.find_elements(By.CLASS_NAME, 'D113_link')
d113_bus_dict={}
for bus in bus_depts:
    for name in popular_bus_names:
        if name in bus.text:
            d113_bus_dict[name]=bus.get_attribute('href')
driver.quit()

Error occurred while processing link: https://www.redbus.in/online-booking/bsrtc-operated-by-vip-travels, skipping... Exception: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16



In [259]:
driver=webdriver.Firefox()
driver.maximize_window()
for name_string, link in d113_bus_dict.items():
    try:
        driver.get(link)
        time.sleep(2)
        
        # Wait for the route elements to be present
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "route"))
        )
        
        try:
            # Check if pagination exists
            parent_div = driver.find_element(By.CLASS_NAME, "DC_117_paginationTable")
            
            try:
                # Try to find pagination buttons
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.DC_117_pageTabs"))
                )
                
                child_count = driver.execute_script(
                    "return arguments[0].getElementsByTagName('div').length;", parent_div
                )
                
                # If buttons are found, iterate through pagination
                for i in range(1, child_count + 1):
                    button = driver.find_element(By.CSS_SELECTOR, f"div.DC_117_pageTabs:nth-child({i})")
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                    driver.execute_script("arguments[0].click();", button)
                    
                    # Extract route details on each page
                    route_details = WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME, 'route'))
                    )
                    for route in route_details:
                        route_link = route.get_attribute('href')
                        travels_links_dict[name_string + '_' + route.text] = route_link
                    
                    time.sleep(1)
                    
            except TimeoutException:
                # If buttons aren't found but parent div exists, get routes from current page
                print(f"Pagination buttons not found for link: {link}. Extracting from current page...")
                route_details = driver.find_elements(By.CLASS_NAME, 'route')
                for route in route_details:
                    route_link = route.get_attribute('href')
                    travels_links_dict[name_string + '_' + route.text] = route_link
                    
        except NoSuchElementException:
            # If pagination table doesn't exist, get routes from current page
            print(f"No pagination found for link: {link}. Extracting from current page...")
            route_details = driver.find_elements(By.CLASS_NAME, 'route')
            for route in route_details:
                route_link = route.get_attribute('href')
                travels_links_dict[name_string + '_' + route.text] = route_link
                
    except Exception as e:
        print(f"Error occurred while processing link: {link}. Skipping... Exception: {e}")
        continue  # Skip to the next link
driver.quit()

Error occurred while processing link: https://www.redbus.in/online-booking/bsrtc-operated-by-vip-travels. Skipping... Exception: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Pagination buttons not found for link: https://www.redbus.in/online-booking/pepsu-punjab. Extracting from current page...


In [260]:
travels_links_dict

{'APSRTC_Hyderabad to Vijayawada': 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada',
 'APSRTC_Vijayawada to Hyderabad': 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad',
 'APSRTC_Hyderabad to Ongole': 'https://www.redbus.in/bus-tickets/hyderabad-to-ongole',
 'APSRTC_Ongole to Hyderabad': 'https://www.redbus.in/bus-tickets/ongole-to-hyderabad',
 'APSRTC_Kakinada to Visakhapatnam': 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam',
 'APSRTC_Bangalore to Tirupati': 'https://www.redbus.in/bus-tickets/bangalore-to-tirupathi',
 'APSRTC_Bangalore to Kadapa': 'https://www.redbus.in/bus-tickets/bangalore-to-kadapa',
 'APSRTC_Hyderabad to Narasaraopet': 'https://www.redbus.in/bus-tickets/hyderabad-to-narsaraopet',
 'APSRTC_Chittoor (Andhra Pradesh) to Bangalore': 'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore',
 'APSRTC_Visakhapatnam to Kakinada': 'https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada',
 'APSRTC_Narasaraopet to Hyd

In [203]:
driver=webdriver.Firefox()
for route_name, route_link in travels_links_dict.items():
    driver.get(route_link)
    driver.maximize_window()
    time.sleep(5)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'button')))
    driver.back()
    body=driver.find_element(By.TAG_NAME,"body")
    view_buses_buttons = [driver.find_element(By.CLASS_NAME, "button")]
    for button in view_buses_buttons:
        button.click()
        time.sleep(1)
    
    scrolling=True
    while scrolling:
        body.send_keys(Keys.CONTROL + Keys.END)
        time.sleep(1)
        if new_page_source==old_page_source:
            scrolling=False
                        
    scrolling=True
    while scrolling:
        old_page_source=driver.page_source
        body.send_keys(Keys.PAGE_UP)
        time.sleep(0.5)
        new_page_source=driver.page_source
        if new_page_source==old_page_source:
            scrolling=False
        details_box = driver.find_elements(By.XPATH, '/html/body/section/div[2]/div[4]/div/div[2]/div/div[2]/div[2]/div[3]/ul/div/li/div[1]/div/div[1]/div[1]')
        for element in details_box:
            bus_name = element.find_element(By.XPATH, ".//div[contains(@class, 'travels')]").text
            bus_type = element.find_element(By.XPATH, ".//div[contains(@class, 'bus-type')]").text
            departing_time = element.find_element(By.XPATH, ".//div[contains(@class, 'dp-time')]").text
            duration = element.find_element(By.XPATH, ".//div[contains(@class, 'dur')]").text
    
    # Combining arrival time and date
            arrival_time = element.find_element(By.XPATH, ".//div[contains(@class, 'bp-time')]").text
            # arrival_date = element.find_element(By.XPATH, ".//div[contains(@class, 'next-day-dp-lbl')]").text
            # reaching_time = f"{arrival_time}"
    
    # Print or process the extracted information
            print(f"Bus Name: {bus_name}")
            print(f"Bus Type: {bus_type}")
            print(f"Departure Time: {departing_time}")
            print(f"Duration: {duration}")
            print(f"Reaching Time: {arrival_time}")
            print("-" * 50)
        

Bus Name: IntrCity SmartBus
Bus Type: A/C Seater / Sleeper (2+1)
Departure Time: 23:05
Duration: 06h 55m
Reaching Time: 06:00 13-Jan
--------------------------------------------------
Bus Name: FRESHBUS
Bus Type: Electric A/C Seater (2+2)
Departure Time: 23:20
Duration: 06h 25m
Reaching Time: 05:45 13-Jan
--------------------------------------------------
Bus Name: FRESHBUS
Bus Type: Electric A/C Seater (2+2)
Departure Time: 22:35
Duration: 06h 30m
Reaching Time: 05:05 13-Jan
--------------------------------------------------
Bus Name: AdSai RK Travels
Bus Type: Non A/C Seater / Sleeper (2+1)
Departure Time: 21:30
Duration: 07h 00m
Reaching Time: 04:30 13-Jan
--------------------------------------------------


NoSuchElementException: Message: Unable to locate element: .//div[contains(@class, 'next-day-dp-lbl')]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [201]:
driver=webdriver.Firefox()
driver.get("https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada")
driver.maximize_window()
time.sleep(5)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'button')))
driver.back()
body=driver.find_element(By.TAG_NAME,"body")
body.send_keys(Keys.PAGE_DOWN)
time.sleep(2)
details_box = driver.find_elements(By.XPATH, '/html/body/section/div[2]/div[4]/div/div[2]/div/div[2]/div[2]/div[3]/ul/div/li/div[1]/div/div[1]/div[1]')

In [202]:
for element in details_box:
    # Using relative XPaths within each element
    bus_name = element.find_element(By.XPATH, ".//div[contains(@class, 'travels')]").text
    bus_type = element.find_element(By.XPATH, ".//div[contains(@class, 'bus-type')]").text
    departing_time = element.find_element(By.XPATH, ".//div[contains(@class, 'dp-time')]").text
    duration = element.find_element(By.XPATH, ".//div[contains(@class, 'dur')]").text
    
    # Combining arrival time and date
    arrival_time = element.find_element(By.XPATH, ".//div[contains(@class, 'bp-time')]").text
    arrival_date = element.find_element(By.XPATH, ".//div[contains(@class, 'next-day-dp-lbl')]").text
    reaching_time = f"{arrival_time} {arrival_date}"
    
    # Print or process the extracted information
    print(f"Bus Name: {bus_name}")
    print(f"Bus Type: {bus_type}")
    print(f"Departure Time: {departing_time}")
    print(f"Duration: {duration}")
    print(f"Reaching Time: {reaching_time}")
    print("-" * 50)

Bus Name: IntrCity SmartBus
Bus Type: A/C Seater / Sleeper (2+1)
Departure Time: 23:05
Duration: 06h 55m
Reaching Time: 06:00 13-Jan
--------------------------------------------------
Bus Name: FRESHBUS
Bus Type: Electric A/C Seater (2+2)
Departure Time: 23:20
Duration: 06h 25m
Reaching Time: 05:45 13-Jan
--------------------------------------------------
Bus Name: FRESHBUS
Bus Type: Electric A/C Seater (2+2)
Departure Time: 22:35
Duration: 06h 30m
Reaching Time: 05:05 13-Jan
--------------------------------------------------
Bus Name: AdYolo Bus
Bus Type: Bharat Benz A/C Sleeper (2+1)
Departure Time: 22:55
Duration: 05h 34m
Reaching Time: 04:29 13-Jan
--------------------------------------------------


NoSuchElementException: Message: Unable to locate element: .//div[contains(@class, 'next-day-dp-lbl')]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16
